# Symphony: CIFAR-10 Example

**Visualizing a [DeepView](https://apple.github.io/deepview/) [Dataset Report](https://apple.github.io/deepview/introspectors/data_introspection/dataset_report.html)**

This is an example of visualizing the CIFAR-10 dataset with Symphony. Beyond the image samples themselves, we've used [DeepView](https://apple.github.io/deepview/) to compute some other statistics about the data. Symphony uses this data in the Familiarity and Duplicates widgets.

In DeepView, you can create a `DatasetReport` object, that has a `data` field, which is a pandas DataFrame table with metadata about each data sample like its familiarity, duplicates, overall summary, and dimensionality projection coordinates. Symphony can directly visualize this DataFrame.

For this example, we'll load a precomputed analysis for the CIFAR-10 dataset that has been saved to disk as a pandas DataFrame. If you are interested in generating this DataFrame yourself (or for a different dataset or model), see [this DeepView example](https://apple.github.io/deepview/notebooks/data_introspection/dataset_report.ipynb). This Symphony example picks up at the end of it.

## Symphony in Jupyter Notebooks

Let's use Symphony to explore this dataset in a Jupyter notebook.

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import cv2
from keras.datasets import cifar10

2024-11-29 01:30:06.884070: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 01:30:06.903993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 01:30:06.904016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 01:30:06.904700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 01:30:06.908546: I tensorflow/core/platform/cpu_feature_guar

Let's first load and download the CIFAR-10 dataset. We'll save it to a folder named `cifar`. 

In [2]:
data_path = "./cifar/"

In [3]:
# Load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
class_to_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Concatenate the train and test into one array, as well as the train/test labels, and the class labels
full_dataset = np.concatenate((x_train, x_test))
dataset_labels = ['train']*len(x_train) + ['test']*len(x_test)
class_labels = np.squeeze(np.concatenate((y_train, y_test)))

# Helper function for file pathing
def class_path(index, dataset_labels, class_labels):
    return f"{dataset_labels[index]}/{class_to_name[int(class_labels[index])]}"

In [4]:
# Loop through data and save images to `cifar` folder
for idx in range(full_dataset.shape[0]):
    base_path = os.path.join(data_path, class_path(idx, dataset_labels, class_labels))
    Path(base_path).mkdir(exist_ok=True, parents=True)
    filename = os.path.join(base_path, f"image{idx}.png")
    # Write to disk after converting to BGR format, used by opencv
    cv2.imwrite(filename, cv2.cvtColor(full_dataset[idx, ...], cv2.COLOR_RGB2BGR))

Now that we have the images saved, we can load our precomputed analysis from DeepView to visualize in Symphony. You can use Symphony to visualize CIFAR-10, and other datsets, directly. But some components require special metadata that we can use DeepView's Dataset Report to generate automatically for us.

We can also print out the DataFrame to see the types of metadata columns that are included.

In [5]:
df = pd.read_pickle('canvas_cifar_example.pkl')

df

,id,class,dataset,duplicates_conv_pw_13,projection_conv_pw_13_x,projection_conv_pw_13_y,familiarity_conv_pw_13,splitFamiliarity_conv_pw_13_byAttr_class,splitFamiliarity_conv_pw_13_byAttr_dataset
0,train/frog/image0.png,frog,train,-1,8.996793,6.067759,-63.839685,"{'frog': -57.4634189968481, 'truck': -79.73388...","{'train': -63.85114012943654, 'test': -63.7891..."
1,train/truck/image1.png,truck,train,-1,0.987265,4.231816,-56.638484,"{'frog': -63.61311723243215, 'truck': -52.5701...","{'train': -56.667391185887446, 'test': -56.511..."
2,train/truck/image2.png,truck,train,-1,3.857024,5.526045,-63.064532,"{'frog': -66.21848904767097, 'truck': -61.3058...","{'train': -63.04006769721045, 'test': -63.1636..."
3,train/deer/image3.png,deer,train,-1,11.347506,4.376108,-70.230302,"{'frog': -77.75308621492171, 'truck': -87.4106...","{'train': -70.17086035521375, 'test': -70.4808..."
4,train/automobile/image4.png,automobile,train,-1,-1.506487,2.882784,-73.770091,"{'frog': -85.30263297363926, 'truck': -82.4036...","{'train': -73.6560069777013, 'test': -74.43546..."
...,...,...,...,...,...,...,...,...,...
59995,test/ship/image59995.png,ship,test,-1,6.271036,3.284976,-70.541111,"{'frog': -77.07471681867696, 'truck': -89.7610...","{'train': -70.54052171442682, 'test': -70.5416..."
59996,test/cat/image59996.png,cat,test,-1,8.761815,3.019561,-67.849741,"{'frog': -70.8622539466022, 'truck': -78.70688...","{'train': -67.81392350569409, 'test': -68.0436..."
59997,test/dog/image59997.png,dog,test,-1,6.577181,3.340236,-68.982097,"{'frog': -75.76836969670552, 'truck': -94.0260...","{'train': -68.97238709302465, 'test': -69.0070..."
59998,test/automobile/image59998.png,automobile,test,-1,-1.076519,2.717594,-68.713629,"{'frog': -78.05870141733367, 'truck': -75.7781...","{'train': -68.68352653575359, 'test': -68.8824..."


To use Symphony, we'll import the main library and instantiate a Symphony object, passing the pandas DataFrame analysis and a file path to the dataset we downloaded.

In [6]:
import canvas_ux

symph = canvas_ux.Canvas(df, files_path=str(data_path))

Canvas spect dict value is {'filesPath': 'notebooks/data_introspection/./cifar/', 'dataType': 2, 'instancesPerPage': 40, 'showUnfilteredData': True, 'idColumn': 'id'}


To use the different Symphony widgets, you can import them indepdently. Let's first look at the Summary widget to see the overall distributions of our datset.

In [7]:
from canvas_summary import CanvasSummary

symph.widget(CanvasSummary)

Canvas spect dict value is {'width': 'XXL', 'height': 'M', 'page': 'Summary', 'name': 'CanvasSummary', 'description': 'A Canvas component that visualizes an overview of a dataset', 'summaryElements': []}


Instead of a summary, if we want to browse through the data we can use the List widget.

In [8]:
from canvas_list import CanvasList

symph.widget(CanvasList)

/media/satish/Development/workspace/projects/deepview_dev/notebooks/data_introspection
Canvas spect dict value is {'width': 'XXL', 'height': 'M', 'page': 'List', 'name': 'CanvasList', 'description': 'A Canvas component that displays a view of data instances'}


It's common to use dimensionality reduction techniques to summarize and find patterns in ML dataset. DeepView already ran a reduction, and saves it when running a DataSet Report. We can use the Scatterplot widget to visualize this embedding.

In [9]:
from canvas_scatterplot import CanvasScatterplot

symph.widget(CanvasScatterplot)

Canvas spect dict value is {'width': 'XXL', 'height': 'M', 'page': 'Scatterplot', 'name': 'CanvasScatterplot', 'description': 'A scatterplot Canvas component based on regl-scatterplot'}


Some datasets can contain duplicates: data instances that are the same or very similar to others. These can be hard to find, and become espeically problematic if the same data instance is in the training and testing splits. We can answer these questions using the Duplicates widget.

Hint: Take a look at the `automobile` class, where there are duplicates across train and test data!

In [10]:
from canvas_duplicates import CanvasDuplicates

symph.widget(CanvasDuplicates)

Canvas spect dict value is {'width': 'XXL', 'height': 'M', 'page': 'Duplicates', 'name': 'CanvasDuplicates', 'description': 'A Canvas component for inspecting potential duplicates in a dataset'}


Lastly, we can use advanced ML metrics and the Familiarity widget to find the most and least representative data instances from a given datset, which can help identify model biases and annotation errors.

In [11]:
from canvas_familiarity import CanvasFamiliarity

symph.widget(CanvasFamiliarity)

Canvas spect dict value is {'width': 'XXL', 'height': 'M', 'page': 'Familiarity', 'name': 'CanvasFamiliarity', 'description': 'A Canvas component to find outliers and common instances in a dataset'}


## Symphony as a Standalone Export

Symphony can also be exported as a standalone static export to be shared with others or hosted. To explore this example in a web browser, you can export the report to local folder.

If you only want to visualize locally without sharing the data, you can specify Symphony to handle the paths for a local standlone visualization by setting ``symlink_files`` to True:

In [ ]:
symph.export('./symphony_report', name="Symphony CIFAR-10 Example", symlink_files=True)

You can now serve the dataset report. For example, from the `symphony_export` folder, run a simple server from the command line:

```bash
python -m http.server
```

And navigate to http://localhost:8000/.